In [22]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.python.framework import constant_op
from tensorflow.python.framework import dtypes
from tensorflow.python.ops import array_ops
from tensorflow.python.ops import math_ops
import pandas as pd
import matplotlib.pyplot as plt
from zipfile import ZipFile
import os

In [10]:
uri = "https://storage.googleapis.com/tensorflow/tf-keras-datasets/jena_climate_2009_2016.csv.zip"
zip_path = keras.utils.get_file(origin=uri, fname="jena_climate_2009_2016.csv.zip")
zip_file = ZipFile(zip_path)
zip_file.extractall()
csv_path = "jena_climate_2009_2016.csv"

df = pd.read_csv(csv_path)

13582336/13568290 [==============================] - 1s 0us/step


In [14]:
titles = [
    "Pressure",
    "Temperature",
    "Temperature in Kelvin",
    "Temperature (dew point)",
    "Relative Humidity",
    "Saturation vapor pressure",
    "Vapor pressure",
    "Vapor pressure deficit",
    "Specific humidity",
    "Water vapor concentration",
    "Airtight",
    "Wind speed",
    "Maximum wind speed",
    "Wind direction in degrees",
]

feature_keys = [
    "p (mbar)",
    "T (degC)",
    "Tpot (K)",
    "Tdew (degC)",
    "rh (%)",
    "VPmax (mbar)",
    "VPact (mbar)",
    "VPdef (mbar)",
    "sh (g/kg)",
    "H2OC (mmol/mol)",
    "rho (g/m**3)",
    "wv (m/s)",
    "max. wv (m/s)",
    "wd (deg)",
]

colors = [
    "blue",
    "orange",
    "green",
    "red",
    "purple",
    "brown",
    "pink",
    "gray",
    "olive",
    "cyan",
]

date_time_key = "Date Time"

In [12]:
split_fraction = 0.715
train_split = int(split_fraction * int(df.shape[0]))
step = 6

past = 720
future = 72
learning_rate = 0.001
batch_size = 256
epochs = 10


def normalize(data, train_split):
    data_mean = data[:train_split].mean(axis=0)
    data_std = data[:train_split].std(axis=0)
    return (data - data_mean) / data_std

In [15]:
print(
    "The selected parameters are:",
    ", ".join([titles[i] for i in [0, 1, 5, 7, 8, 10, 11]]),
)
selected_features = [feature_keys[i] for i in [0, 1, 5, 7, 8, 10, 11]]
features = df[selected_features]
features.index = df[date_time_key]
features.head()

features = normalize(features.values, train_split)
features = pd.DataFrame(features)
features.head()

train_data = features.loc[0 : train_split - 1]
val_data = features.loc[train_split:]

The selected parameters are: Pressure, Temperature, Saturation vapor pressure, Vapor pressure deficit, Specific humidity, Airtight, Wind speed


In [16]:
start = past + future
end = start + train_split

x_train = train_data[[i for i in range(7)]].values
y_train = features.iloc[start:end][[1]]

sequence_length = int(past / step)

In [17]:
dataset_train = keras.preprocessing.timeseries_dataset_from_array(
    x_train,
    y_train,
    sequence_length=sequence_length,
    sampling_rate=step,
    batch_size=batch_size,
)

In [18]:
x_end = len(val_data) - past - future

label_start = train_split + past + future

x_val = val_data.iloc[:x_end][[i for i in range(7)]].values
y_val = features.iloc[label_start:][[1]]

dataset_val = keras.preprocessing.timeseries_dataset_from_array(
    x_val,
    y_val,
    sequence_length=sequence_length,
    sampling_rate=step,
    batch_size=batch_size,
)


for batch in dataset_train.take(1):
    inputs, targets = batch

print("Input shape:", inputs.numpy().shape)
print("Target shape:", targets.numpy().shape)

Input shape: (256, 120, 7)
Target shape: (256, 1)


Training with ANN.

In [38]:
inputs = keras.layers.Input(shape=(inputs.shape[1], inputs.shape[2]))
lstm_out = Linear(4)(inputs)
outputs = keras.layers.Dense(1)(lstm_out)

model = keras.Model(inputs=inputs, outputs=outputs)
model.compile(optimizer=keras.optimizers.Adam(learning_rate=learning_rate), loss="mse")
model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_9 (InputLayer)        [(None, 120, 7)]          0         
                                                                 
 linear_2 (Linear)           (None, 120, 4)            32        
                                                                 
 dense_4 (Dense)             (None, 120, 1)            5         
                                                                 
Total params: 37
Trainable params: 37
Non-trainable params: 0
_________________________________________________________________


In [39]:
model.fit(
    dataset_train,
    epochs=epochs,
    validation_data=dataset_val
)

Epoch 1/10
1172/1172 [==============================] - 10s 8ms/step - loss: 0.4818 - val_loss: 78.2747
Epoch 2/10
1172/1172 [==============================] - 9s 8ms/step - loss: 0.3956 - val_loss: 81.6294
Epoch 3/10
1172/1172 [==============================] - 9s 8ms/step - loss: 0.3906 - val_loss: 85.3871
Epoch 4/10
1172/1172 [==============================] - 9s 8ms/step - loss: 0.3887 - val_loss: 86.5176
Epoch 5/10
1172/1172 [==============================] - 10s 9ms/step - loss: 0.3878 - val_loss: 86.4181
Epoch 6/10
1172/1172 [==============================] - 9s 8ms/step - loss: 0.3872 - val_loss: 85.7269
Epoch 7/10
1172/1172 [==============================] - 9s 8ms/step - loss: 0.3869 - val_loss: 84.7561
Epoch 8/10
1172/1172 [==============================] - 9s 8ms/step - loss: 0.3866 - val_loss: 83.6640
Epoch 9/10
1172/1172 [==============================] - 9s 8ms/step - loss: 0.3863 - val_loss: 82.5327
Epoch 10/10
1172/1172 [==============================] - 9s 8ms/step - 

Training with LSTM.

In [40]:
inputs = keras.layers.Input(shape=(inputs.shape[1], inputs.shape[2]))
lstm_out, _ = LSTM(4)(inputs)
outputs = keras.layers.Dense(1)(lstm_out)

model = keras.Model(inputs=inputs, outputs=outputs)
model.compile(optimizer=keras.optimizers.Adam(learning_rate=learning_rate), loss="mse")
model.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_10 (InputLayer)       [(None, 120, 7)]          0         
                                                                 
 lstm_6 (LSTM)               ((None, 120, 4),          192       
                              (None, 240, 4))                    
                                                                 
 dense_5 (Dense)             (None, 120, 1)            5         
                                                                 
Total params: 197
Trainable params: 197
Non-trainable params: 0
_________________________________________________________________


In [41]:
model.fit(
    dataset_train,
    epochs=epochs,
    validation_data=dataset_val
)

Epoch 1/10
1172/1172 [==============================] - 12s 10ms/step - loss: 0.5135 - val_loss: 0.3594
Epoch 2/10
1172/1172 [==============================] - 11s 9ms/step - loss: 0.3864 - val_loss: 0.3645
Epoch 3/10
1172/1172 [==============================] - 11s 10ms/step - loss: 0.3831 - val_loss: 0.3641
Epoch 4/10
1172/1172 [==============================] - 11s 9ms/step - loss: 0.3814 - val_loss: 0.3629
Epoch 5/10
1172/1172 [==============================] - 12s 10ms/step - loss: 0.3801 - val_loss: 0.3618
Epoch 6/10
1172/1172 [==============================] - 11s 9ms/step - loss: 0.3789 - val_loss: 0.3607
Epoch 7/10
1172/1172 [==============================] - 11s 9ms/step - loss: 0.3778 - val_loss: 0.3596
Epoch 8/10
1172/1172 [==============================] - 11s 10ms/step - loss: 0.3768 - val_loss: 0.3587
Epoch 9/10
1172/1172 [==============================] - 11s 9ms/step - loss: 0.3759 - val_loss: 0.3579
Epoch 10/10
1172/1172 [==============================] - 11s 9ms/step

In [43]:
inputs = keras.layers.Input(shape=(inputs.shape[1], inputs.shape[2]))
lstm_out, _ = LSTM(4, trans=True, iters=5)(inputs)
outputs = keras.layers.Dense(1)(lstm_out)

model = keras.Model(inputs=inputs, outputs=outputs)
model.compile(optimizer=keras.optimizers.Adam(learning_rate=learning_rate), loss="mse")
model.summary()

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_11 (InputLayer)       [(None, 120, 7)]          0         
                                                                 
 lstm_7 (LSTM)               ((None, 120, 4),          248       
                              (None, 240, 4))                    
                                                                 
 dense_6 (Dense)             (None, 120, 1)            5         
                                                                 
Total params: 253
Trainable params: 253
Non-trainable params: 0
_________________________________________________________________


In [44]:
model.fit(
    dataset_train,
    epochs=epochs,
    validation_data=dataset_val
)

Epoch 1/10
1172/1172 [==============================] - 18s 15ms/step - loss: 0.5624 - val_loss: 0.3574
Epoch 2/10
1172/1172 [==============================] - 17s 14ms/step - loss: 0.3912 - val_loss: 0.3646
Epoch 3/10
1172/1172 [==============================] - 17s 15ms/step - loss: 0.3846 - val_loss: 0.3638
Epoch 4/10
1172/1172 [==============================] - 17s 15ms/step - loss: 0.3817 - val_loss: 0.3628
Epoch 5/10
1172/1172 [==============================] - 17s 15ms/step - loss: 0.3799 - val_loss: 0.3619
Epoch 6/10
1172/1172 [==============================] - 17s 14ms/step - loss: 0.3787 - val_loss: 0.3611
Epoch 7/10
1172/1172 [==============================] - 17s 15ms/step - loss: 0.3777 - val_loss: 0.3603
Epoch 8/10
1172/1172 [==============================] - 16s 14ms/step - loss: 0.3770 - val_loss: 0.3595
Epoch 9/10
1172/1172 [==============================] - 17s 14ms/step - loss: 0.3763 - val_loss: 0.3587
Epoch 10/10
1172/1172 [==============================] - 17s 14m

# Code Repo

### Code Snippet A: Adam Optimizer.

In [2]:
opt = tf.keras.optimizers.Adam(learning_rate=0.1)
var1 = tf.Variable(10.0)
loss = lambda: (var1 ** 2)/2.0       # d(loss)/d(var1) == var1
step_count = opt.minimize(loss, [var1]).numpy()
# The first step is `-learning_rate*sign(grad)`
var1.numpy()

9.9

### Code Snippet B: Layer Interface.

In [4]:
class Linear(keras.layers.Layer):
    def __init__(self, units=32):
        super(Linear, self).__init__()
        self.units = units

    def build(self, input_shape):
        self.w = self.add_weight(
            shape=(input_shape[-1], self.units),
            initializer="random_normal",
            trainable=True,
        )
        self.b = self.add_weight(
            shape=(self.units,), initializer="random_normal", trainable=True
        )

    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b

### Code Snippet C: Naive LSTM

In [42]:
'''
A Naive LSTM Cell Implementation.

@link tensorflow/tensorflow/python/keras/layers/legacy_rnn/rnn_cell_impl.py#BasicLSTMCell
@link https://github.com/piEsposito/pytorch-lstm-by-hand/blob/master/LSTM.ipynb
@date MAR-29-2022
@note straightforward, in sacrifice of efficiency
'''
class LSTM(keras.layers.Layer):
    def __init__(self, units=32, trans=False, iters=3):
        super(LSTM, self).__init__()
        self.units = units
        self.trans = trans
        self.iters = iters

    def build(self, input_shape):
        self.W_i = self.add_weight(shape=(input_shape[-1], self.units), initializer="random_normal", trainable=True)
        self.U_i = self.add_weight(shape=(self.units, self.units), initializer="random_normal", trainable=True)
        self.b_i = self.add_weight(shape=(self.units, ), initializer="random_normal", trainable=True)

        self.W_f = self.add_weight(shape=(input_shape[-1], self.units), initializer="random_normal", trainable=True)
        self.U_f = self.add_weight(shape=(self.units, self.units), initializer="random_normal", trainable=True)
        self.b_f = self.add_weight(shape=(self.units, ), initializer="random_normal", trainable=True)

        self.W_c = self.add_weight(shape=(input_shape[-1], self.units), initializer="random_normal", trainable=True)
        self.U_c = self.add_weight(shape=(self.units, self.units), initializer="random_normal", trainable=True)
        self.b_c = self.add_weight(shape=(self.units, ), initializer="random_normal", trainable=True)

        self.W_o = self.add_weight(shape=(input_shape[-1], self.units), initializer="random_normal", trainable=True)
        self.U_o = self.add_weight(shape=(self.units, self.units), initializer="random_normal", trainable=True)
        self.b_o = self.add_weight(shape=(self.units, ), initializer="random_normal", trainable=True)

        if self.trans:
            self.Q   = self.add_weight(shape=(self.units, input_shape[-1]), initializer="random_normal", trainable=True)
            self.R   = self.add_weight(shape=(input_shape[-1], self.units), initializer="random_normal", trainable=True)

    def mogrigy(self, x_t, h_t):
        sigmoid = math_ops.sigmoid
        for i in range(1, self.iters + 1):
            if (i % 2 == 0):
                h_t = (2 * sigmoid(x_t @ self.R)) * h_t
            else:
                x_t = (2 * sigmoid(h_t @ self.Q)) * x_t
        return x_t, h_t

    def call(self, inputs, init_states=None):
        bs = inputs.shape[1]
        one = constant_op.constant(1, dtype=dtypes.int32)
        sigmoid = math_ops.sigmoid
        tanh = math_ops.tanh
        if init_states is None:
            h_t, c_t = (
                tf.zeros((bs, self.units)),
                tf.zeros((bs, self.units)),
            )
        else:
            h_t, c_t = array_ops.split(value=init_states, num_or_size_splits=2, axis=one)

        x_t = inputs

        if self.trans:
            x_t, h_t = self.mogrigy(x_t, h_t)

        i_t = sigmoid(x_t @ self.W_i + h_t @ self.U_i + self.b_i)
        f_t = sigmoid(x_t @ self.W_f + h_t @ self.U_f + self.b_f)
        g_t = tanh(x_t @ self.W_c + h_t @ self.U_c + self.b_c)
        o_t = sigmoid(x_t @ self.W_o + h_t @ self.U_o + self.b_o)
        c_t = f_t * c_t + i_t * g_t
        h_t = o_t * tanh(c_t)
            
        new_state = array_ops.concat([h_t, c_t], 1)
        return h_t, new_state